<a href="https://colab.research.google.com/github/Nandhini-s25/sdcproject/blob/main/randomforestemperature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install Gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.5 MB/s eta 0:00:00


In [9]:
import numpy as np
import pandas as pd

# Generate synthetic customer data
num_samples = 1000
data = {
    "tenure": np.random.randint(1, 60, num_samples),
    "MonthlyCharges": np.random.randint(500, 5000, num_samples),
    "TotalCharges": lambda df: df["tenure"] * df["MonthlyCharges"],
    "Contract": np.random.choice([0, 1], num_samples),  # 0 = Month-to-Month, 1 = Yearly
    "Churn": np.random.choice([0, 1], num_samples, p=[0.7, 0.3])  # 70% stay, 30% churn
}

df = pd.DataFrame(data)
df["TotalCharges"] = df.apply(lambda row: row["tenure"] * row["MonthlyCharges"], axis=1)

# Save dataset
df.to_csv("synthetic_churn_data.csv", index=False)
print("Synthetic dataset created!")

Synthetic dataset created!


In [15]:
import numpy as np
import pandas as pd
import gradio as gr
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler

# Step 1: Generate Balanced Weather Data
num_samples = 1000
np.random.seed(42)

data = {
    "Temperature": np.random.randint(10, 40, num_samples),
    "Humidity": np.random.randint(20, 100, num_samples),
    "WindSpeed": np.random.randint(0, 20, num_samples),
    "Weather": np.random.choice(["Sunny", "Rainy", "Cloudy"], num_samples, p=[0.33, 0.33, 0.34])
}

df = pd.DataFrame(data)

# Encode target variable
df["Weather"] = df["Weather"].map({"Sunny": 0, "Rainy": 1, "Cloudy": 2})

# Step 2: Prepare Features & Target
X = df.drop(columns=["Weather"])
y = df["Weather"]

# Step 3: Split Data (Train/Test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 4: Standardize Features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 5: Train Optimized Random Forest Model
model = RandomForestClassifier(n_estimators=200, max_depth=8, criterion="entropy", random_state=42)
model.fit(X_train, y_train)

# Step 6: Define Prediction Function Using Probabilities
def weather_prediction(temp, humidity, wind_speed):
    input_data = np.array([[temp, humidity, wind_speed]])
    input_scaled = scaler.transform(input_data)
    probabilities = model.predict_proba(input_scaled)[0]
    predicted_class = np.argmax(probabilities)
    return {0: "Cloudy ☁️", 1: "Rainy 🌧️", 2: "Sunny ☀️"}[predicted_class]

# Step 7: Create Gradio UI
interface = gr.Interface(
    fn=weather_prediction,
    inputs=[
        gr.Number(label="Temperature (°C)"),
        gr.Number(label="Humidity (%)"),
        gr.Number(label="Wind Speed (km/h)"),
    ],
    outputs=gr.Label(),
    title="Weather Prediction AI (Optimized Random Forest)",
    description="Enter weather conditions to predict if it's sunny, rainy, or cloudy.",
)

# Step 8: Launch the Web App
interface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7aecb2df1d6904ea17.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
